In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os
from imblearn.over_sampling import SMOTE
import joblib

from google.colab import drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/NF-CSE-CIC-IDS2018.csv')

# Check dataset shape and for missing/duplicate values
print(data.shape)
print(data.isnull().sum())
print(data.duplicated().sum())

# Drop duplicate rows
data.drop_duplicates(inplace=True)

# Separate features and target variable
x = data.drop('Label', axis=1)
y = data['Label']
categorical_features = x.select_dtypes(include=['object']).columns

# Label encoding for categorical features
label_encoder = LabelEncoder()
for feature in categorical_features:
    x[feature] = label_encoder.fit_transform(x[feature])
y_encoded = label_encoder.fit_transform(y)

# Split data into training, validation, and test sets
x_train, x_test, y_train_encoded, y_test_encoded = train_test_split(x, y_encoded, test_size=0.2, random_state=42)
x_train, x_val, y_train_encoded, y_val_encoded = train_test_split(x_train, y_train_encoded, test_size=0.25, random_state=42)

# Apply standard scaling to numerical features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_scaled, y_train_encoded)

# Train the RandomForest model
model = RandomForestClassifier(random_state=42)
model.fit(x_train_resampled, y_train_resampled)

# Save the trained model to disk
model_file_path = "/content/drive/MyDrive/nids_model.pkl"
joblib.dump(model, model_file_path)

# Evaluate the model on the validation set
y_pred_val_encoded = model.predict(x_val_scaled)
accuracy_val = accuracy_score(y_val_encoded, y_pred_val_encoded)
print(f"Validation Accuracy: {accuracy_val}")

# Evaluate the model on the test set
y_pred_test_encoded = model.predict(x_test_scaled)
accuracy_test = accuracy_score(y_test_encoded, y_pred_test_encoded)
print(f"Test Accuracy: {accuracy_test}")

# Ensure the model is saved
if os.path.exists(model_file_path):
    print("Model saved successfully and ready for real-time detection.")
else:
    print("Error: Model file not found!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(8392401, 14)
IPV4_SRC_ADDR                 0
L4_SRC_PORT                   0
IPV4_DST_ADDR                 0
L4_DST_PORT                   0
PROTOCOL                      0
L7_PROTO                      0
IN_BYTES                      0
OUT_BYTES                     0
IN_PKTS                       0
OUT_PKTS                      0
TCP_FLAGS                     0
FLOW_DURATION_MILLISECONDS    0
Label                         0
Attack                        0
dtype: int64
1077740
Validation Accuracy: 1.0
Test Accuracy: 1.0
Model saved successfully and ready for real-time detection.
